In [88]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")

In [89]:
from pandas import pandas as pd

data_train = pd.read_json("dataset-train-spm.json")
data_train["ids"] = data_train["message"].apply(lambda x: tokenizer.encode(x).ids)

X_train = data_train["ids"]
Y_train = data_train["type"]
tokenizer.get_vocab_size()

102400

In [90]:
data_train

,message,type,ids
0,강릉축협하나로마트 입암점 - - 강릉축협하나로마트 입암점밴드가입: https://b...,0,"[43656, 3236, 7059, 7027, 715, 5285, 1110, 488..."
1,지방시 뷰티 샘플링 및 메이크업 서비스 이벤트칙칙하고 생기 잃은 눈가 피부를 개선해...,0,"[20271, 954, 13547, 19006, 3890, 2425, 12425, ..."
2,<< ln.run/vb_ z >>적어 가입시교촌무슨이유든노재쟤뉴비 % all 죵목 ...,1,"[35642, 17146, 375, 24847, 376, 38555, 424, 17..."
3,언뉘들 많은 곳~이쁜이들이 기다리고 있어요연락주세요 !!llne:x x,1,"[2833, 78491, 1371, 1180, 732, 1763, 455, 3673..."
4,손흥민 응원만 하 면 3 만원 톳넘 5 처넌 까고 흥민이 골 넣으면 배달의민족 3만...,1,"[75981, 11204, 1043, 691, 928, 582, 380, 926, ..."
...,...,...,...
14532,오창정육점 / . 일요일 정상영업 합니다 만원 정육 구매시 선물 증정 합니다한돈 암...,0,"[1235, 3239, 1002, 2291, 1647, 841, 582, 375, ..."
14533,cure-tv 주말 신규 방문 편의점 쿠폰 5만추가지급 v카카오v id : xen44,1,"[428, 1025, 374, 14437, 5677, 10145, 3325, 141..."
14534,<< 마포 에피트 어바닉 >> 최고경쟁률 : 잔여세대 선착순 줍줍!!# 입지환경 호...,0,"[35642, 23286, 26201, 1067, 767, 1987, 8777, 1..."
14535,"[하나은행] 년 은행달력 증정 이벤트 배 현님! 년 은행달력, 전담직원이 다시 한번...",0,"[420, 7027, 9906, 422, 1393, 5610, 2230, 1398,..."


In [91]:
from tf_keras.preprocessing.sequence import pad_sequences

max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)

In [92]:
from tf_keras.layers import SimpleRNN, LSTM, Embedding, Dense
from tf_keras.models import Sequential
from tf_keras.callbacks import EarlyStopping, ModelCheckpoint

# 32,16: 0.974 0.987
# 32,32: 0.979 0.985
# 64,32:
hidden_units = 64
embedding_dim = 32

model = Sequential()
model.add(Embedding(tokenizer.get_vocab_size(), embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('rnn-spm.keras', monitor = 'val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[es, mc])


Epoch 1/10
182/182 [==============================] - ETA: 0s - loss: 0.2533 - acc: 0.8909
Epoch 1: val_acc improved from -inf to 0.97593, saving model to rnn-spm.keras
182/182 [==============================] - 125s 601ms/step - loss: 0.2533 - acc: 0.8909 - val_loss: 0.0766 - val_acc: 0.9759
Epoch 2/10
182/182 [==============================] - ETA: 0s - loss: 0.0592 - acc: 0.9813
Epoch 2: val_acc improved from 0.97593 to 0.98487, saving model to rnn-spm.keras
182/182 [==============================] - 95s 520ms/step - loss: 0.0592 - acc: 0.9813 - val_loss: 0.0424 - val_acc: 0.9849
Epoch 3/10
182/182 [==============================] - ETA: 0s - loss: 0.0345 - acc: 0.9912
Epoch 3: val_acc improved from 0.98487 to 0.98556, saving model to rnn-spm.keras
182/182 [==============================] - 81s 444ms/step - loss: 0.0345 - acc: 0.9912 - val_loss: 0.0395 - val_acc: 0.9856
Epoch 4/10
182/182 [==============================] - ETA: 0s - loss: 0.0245 - acc: 0.9935
Epoch 4: val_acc did no